In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# --- Load Data ---
orders_data_path = "C:/Users/parij/OneDrive/Desktop/git_practice/UnORG_supply_chain_ml/order_data_last_six_month.xlsx"
items_data_path = "C:/Users/parij/OneDrive/Desktop/git_practice/UnORG_supply_chain_ml/associated_order_item_data_last_six_month.xlsx"

orders = pd.read_excel(orders_data_path)
order_items = pd.read_excel(items_data_path)
# Merge them on order_id
merged = pd.merge(order_items, orders, on='order_id', how='inner')

merged['order_date'] = pd.to_datetime(merged['order_date'])

def create_customer_product_features(df):
    """
    Create features describing customer-product relationships
    """
    # Get latest date in dataset for recency calculations
    latest_date = df['order_date'].max()

    # Group by customer-product pairs
    cp_features = df.groupby(['customer_id', 'item_name']).agg({
        'order_date': [
            'count',  # Number of times ordered
            'max',    # Last order date
            lambda x: (latest_date - x.max()).days  # Recency
        ],
        'quantity': [
            'sum',  # Total quantity ordered
            'mean', # Average order size
            'std'   # Variability in order size
        ]
    })

    # Flatten column names
    cp_features.columns = ['_'.join(col).strip('_') for col in cp_features.columns.values]
    cp_features = cp_features.reset_index()

    # Rename columns for clarity
    cp_features.rename(columns={
        'order_date_count': 'order_frequency',
        'order_date_max': 'last_order_date',
        'order_date_<lambda_0>': 'days_since_last_order',
        'quantity_sum': 'total_quantity',
        'quantity_mean': 'avg_quantity',
        'quantity_std': 'quantity_std'
    }, inplace=True)

    # Fill missing values for std (occurs when only ordered once)
    cp_features['quantity_std'] = cp_features['quantity_std'].fillna(0)

    return cp_features

# Apply the function to create customer-product features
customer_product_features = create_customer_product_features(merged)
print("\nCustomer-Product Features:")
print(customer_product_features.head())

In [ ]:
def add_customer_features(df, cp_features):
    """
    Add customer-level features to enhance predictions
    """
    # Create customer-level aggregations
    customer_features = df.groupby('customer_id').agg({
        'order_id': 'nunique',  # Number of orders
        'item_name': 'nunique', # Number of unique products
        'quantity': 'sum',      # Total quantity ordered
        'order_date': ['min', 'max', 'nunique']  # First, last, and number of order dates
    })

    # Flatten column names
    customer_features.columns = ['_'.join(col).strip('_') for col in customer_features.columns.values]
    customer_features = customer_features.reset_index()

    # Calculate average order frequency (days between first and last order ÷ number of orders)
    customer_features['avg_days_between_orders'] = (
        (customer_features['order_date_max'] - customer_features['order_date_min']).dt.days /
        customer_features['order_date_nunique']
    )

    # Fill infinities (if customer only ordered once)
    customer_features['avg_days_between_orders'] = customer_features['avg_days_between_orders'].replace(np.inf, 0)

    # Rename columns
    customer_features.rename(columns={
        'order_id_nunique': 'total_orders',
        'item_name_nunique': 'unique_products',
        'quantity_sum': 'customer_total_quantity',
        'order_date_nunique': 'unique_order_dates'
    }, inplace=True)

    # Merge with customer-product features
    features_df = pd.merge(cp_features, customer_features, on='customer_id', how='left')

    return features_df

# Apply the function to add customer-level features
enhanced_features = add_customer_features(merged, customer_product_features)
print("\nEnhanced Features with Customer Data:")
print(enhanced_features.head())

In [ ]:
def add_product_features(df, features_df):
    """
    Add product-level features for better prediction
    """
    # Create product-level aggregations
    product_features = df.groupby('item_name').agg({
        'customer_id': 'nunique',  # Number of unique customers
        'order_id': 'nunique',     # Number of orders
        'quantity': ['sum', 'mean'] # Total quantity and average quantity
    })

    # Flatten column names
    product_features.columns = ['_'.join(col).strip('_') for col in product_features.columns.values]
    product_features = product_features.reset_index()

    # Calculate product popularity score (weighted by both customer count and order count)
    product_features['popularity_score'] = (
        product_features['customer_id_nunique'] *
        product_features['order_id_nunique']
    ) / product_features['customer_id_nunique'].max() / product_features['order_id_nunique'].max()

    # Rename columns
    product_features.rename(columns={
        'customer_id_nunique': 'customer_count',
        'order_id_nunique': 'order_count',
        'quantity_sum': 'product_total_quantity',
        'quantity_mean': 'product_avg_quantity'
    }, inplace=True)

    # Merge with existing features
    features_df = pd.merge(features_df, product_features, on='item_name', how='left')

    return features_df

# Apply the function to add product-level features
complete_features = add_product_features(merged, enhanced_features)
print("\nComplete Features with Product Data:")
print(complete_features.head())

In [ ]:
def add_interaction_features(features_df):
    """
    Add interaction features that capture combined behavior
    """
    # Calculate how much of a customer's total quantity is this specific product
    features_df['product_share'] = features_df['total_quantity'] / features_df['customer_total_quantity']

    # Calculate order frequency relative to customer's overall ordering pattern
    features_df['relative_order_frequency'] = features_df['order_frequency'] / features_df['total_orders']

    # Customer's tendency to reorder (based on recency and frequency)
    features_df['reorder_score'] = features_df['order_frequency'] / (features_df['days_since_last_order'] + 1)

    # Product affinity (combine order frequency with quantity)
    features_df['product_affinity'] = features_df['order_frequency'] * features_df['total_quantity']

    # Calculate expected days until next order
    features_df['expected_days_to_next_order'] = (
        features_df['avg_days_between_orders'] -
        features_df['days_since_last_order']
    ).clip(lower=0)

    return features_df

# Apply the function to add interaction features
final_features = add_interaction_features(complete_features)
print("\nFinal Features with Interactions:")
print(final_features.head())

In [ ]:
def prepare_training_data(df, features_df, cutoff_date, prediction_window=14):
    """
    Prepare data for training the prediction models
    """
    # Define the target window
    prediction_start = cutoff_date + pd.Timedelta(days=1)
    prediction_end = cutoff_date + pd.Timedelta(days=prediction_window)

    print(f"Preparing to predict orders from {prediction_start} to {prediction_end}")

    # Get actual orders in the target period for training
    target_orders = df[
        (df['order_date'] >= prediction_start) &
        (df['order_date'] <= prediction_end)
    ]

    # Create target variables
    # 1. Was this product ordered by this customer in the prediction window?
    ordered_pairs = target_orders.groupby(['customer_id', 'item_name']).size().reset_index()
    ordered_pairs.columns = ['customer_id', 'item_name', 'order_count']
    ordered_pairs['was_ordered'] = 1

    # 2. How much was ordered in the prediction window?
    quantity_ordered = target_orders.groupby(['customer_id', 'item_name'])['quantity'].sum().reset_index()

    # Merge targets with features
    training_data = pd.merge(
        features_df,
        ordered_pairs[['customer_id', 'item_name', 'was_ordered']],
        on=['customer_id', 'item_name'],
        how='left'
    )

    training_data['was_ordered'] = training_data['was_ordered'].fillna(0)

    # Merge quantity information where applicable
    training_data = pd.merge(
        training_data,
        quantity_ordered,
        on=['customer_id', 'item_name'],
        how='left'
    )

    training_data['quantity'] = training_data['quantity'].fillna(0)

    # Select features for modeling
    feature_cols = [
        'order_frequency', 'days_since_last_order', 'total_quantity',
        'avg_quantity', 'quantity_std', 'total_orders', 'unique_products',
        'customer_total_quantity', 'customer_count', 'order_count',
        'product_total_quantity', 'product_avg_quantity', 'popularity_score',
        'product_share', 'relative_order_frequency', 'reorder_score',
        'product_affinity', 'expected_days_to_next_order'
    ]

    return training_data, feature_cols

# Define a cutoff date for training (using part of the data)
# This should be a date that leaves enough future data for validation
max_date = merged['order_date'].max()
cutoff_date = max_date - pd.Timedelta(days=14)  # Use data up to 14 days before the last date

# Prepare training data
training_data, feature_cols = prepare_training_data(merged, final_features, cutoff_date)
print("\nTraining Data prepared with targets:")
print(training_data.head())

In [ ]:
def build_product_selection_model(training_data, feature_cols):
    """
    Build a model to predict which products a customer will order
    """
    # Prepare features and target for the product selection model
    X = training_data[feature_cols]
    y = training_data['was_ordered']

    # Check class distribution
    class_counts = np.bincount(y.astype(int))
    print(f"Class distribution - Not ordered: {class_counts[0]}, Ordered: {class_counts[1]}")

    # Calculate class weight to handle imbalance
    weight_ratio = class_counts[0] / class_counts[1] if class_counts[1] > 0 else 10
    class_weight = {0: 1, 1: min(weight_ratio, 10)}  # Cap at 10 to avoid extreme weights
    print(f"Using class weights: {class_weight}")

    # Split data for training and validation
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight=class_weight,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_scaled, y_train)

    # Evaluate
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    print("\nProduct Selection Model Evaluation:")
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC: {roc_auc_score(y_test, y_proba):.4f}")

    # Feature importance
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 5 important features:")
    print(importance.head(5))

    return model, scaler

# Build the product selection model
selection_model, selection_scaler = build_product_selection_model(training_data, feature_cols)



In [ ]:
%pip install joblib
import joblib

In [ ]:
joblib.dump(selection_model,'C:/Users/parij/OneDrive/Desktop/git_practice/UnORG_supply_chain_ml/models/selection_model.pkl')

In [ ]:
def build_quantity_prediction_model(training_data, feature_cols):
    """
    Build a model to predict the quantity of each product a customer will order
    """
    # Filter to only products that were ordered (quantity > 0)
    quantity_data = training_data[training_data['was_ordered'] == 1].copy()

    if len(quantity_data) == 0:
        print("No products were ordered in the target period. Cannot build quantity model.")
        return None, None

    # Prepare features and target for the quantity prediction model
    X = quantity_data[feature_cols]
    y = quantity_data['quantity']

    # Split data for training and validation
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train model
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=8,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_scaled, y_train)

    # Evaluate
    y_pred = model.predict(X_test_scaled)

    # Round predictions to nearest integer (can't order partial quantities)
    y_pred_rounded = np.round(y_pred).clip(1)

    print("\nQuantity Prediction Model Evaluation:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred_rounded):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_rounded)):.2f}")
    print(f"R²: {r2_score(y_test, y_pred_rounded):.4f}")

    # Feature importance
    importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 5 important features for quantity prediction:")
    print(importance.head(5))

    return model, scaler

# Build the quantity prediction model
quantity_model, quantity_scaler = build_quantity_prediction_model(training_data, feature_cols)


In [ ]:
joblib.dump(quantity_model,'C:/Users/parij/OneDrive/Desktop/git_practice/UnORG_supply_chain_ml/models/quantity_model.pkl')

In [ ]:
def predict_future_orders(features_df, selection_model, quantity_model,
                         selection_scaler, quantity_scaler, feature_cols, threshold=0.5):
    """
    Predict which products each customer will order and in what quantity
    """
    # Get features for prediction
    X_pred = features_df[feature_cols]

    # Scale features
    X_pred_scaled = selection_scaler.transform(X_pred)

    # Predict product selection probability
    selection_proba = selection_model.predict_proba(X_pred_scaled)[:, 1]

    # Add predictions to features dataframe
    prediction_results = features_df[['customer_id', 'item_name']].copy()
    prediction_results['selection_probability'] = selection_proba
    prediction_results['will_order'] = (selection_proba >= threshold).astype(int)

    # Filter to products predicted to be ordered
    will_order = prediction_results[prediction_results['will_order'] == 1].copy()

    if len(will_order) == 0:
        print("No products predicted to be ordered")
        return prediction_results

    # Predict quantities for selected products
    if quantity_model is not None:
        X_qty_pred = X_pred.loc[will_order.index]
        X_qty_pred_scaled = quantity_scaler.transform(X_qty_pred)

        quantity_pred = quantity_model.predict(X_qty_pred_scaled)
        will_order['predicted_quantity'] = np.round(quantity_pred).clip(1).astype(int)
    else:
        # If no quantity model, use historical average
        will_order['predicted_quantity'] = features_df.loc[will_order.index, 'avg_quantity'].fillna(1).round().clip(1).astype(int)

    # Merge back with full results
    prediction_results = pd.merge(
        prediction_results,
        will_order[['customer_id', 'item_name', 'predicted_quantity']],
        on=['customer_id', 'item_name'],
        how='left'
    )

    prediction_results['predicted_quantity'] = prediction_results['predicted_quantity'].fillna(0).astype(int)

    # Count predicted orders
    orders_count = len(will_order)
    customers_count = will_order['customer_id'].nunique()
    products_count = will_order['item_name'].nunique()

    print(f"\nPredicted {orders_count} orders across {customers_count} customers for {products_count} products")

    return prediction_results

# Generate predictions
predictions = predict_future_orders(
    final_features,
    selection_model,
    quantity_model,
    selection_scaler,
    quantity_scaler,
    feature_cols
)

print("\nSample predictions:")
print(predictions.head(10))


In [ ]:
def aggregate_demand_by_product(predictions):
    """
    Aggregate demand by product for inventory planning
    """
    # Filter to rows with predicted orders
    ordered = predictions[predictions['will_order'] == 1]

    # Group by product
    product_demand = ordered.groupby('item_name').agg({
        'predicted_quantity': 'sum',
        'customer_id': 'nunique'
    }).reset_index()

    product_demand.columns = ['item_name', 'total_predicted_quantity', 'unique_customers']
    product_demand = product_demand.sort_values('total_predicted_quantity', ascending=False)

    print("\nTop products by predicted demand:")
    print(product_demand.head(10))

    # Visualize top products
    plt.figure(figsize=(12, 8))
    top_products = product_demand.head(15)
    sns.barplot(x='total_predicted_quantity', y='item_name', data=top_products)
    plt.title('Top 15 Products by Predicted Demand')
    plt.xlabel('Predicted Quantity')
    plt.ylabel('Product')
    plt.tight_layout()
    plt.show()

    return product_demand

# Aggregate demand by product
product_demand = aggregate_demand_by_product(predictions)


In [ ]:
def generate_customer_recommendations(predictions, top_n=5):
    """
    Generate top product recommendations for each customer
    """
    # Get customers with predictions
    customers = predictions['customer_id'].unique()

    all_recommendations = []

    for customer in customers:
        # Get customer's predictions
        customer_preds = predictions[predictions['customer_id'] == customer].copy()

        # Sort by selection probability
        customer_preds = customer_preds.sort_values('selection_probability', ascending=False)

        # Get top N recommendations
        top_recommendations = customer_preds.head(top_n)

        # Only include products with non-zero probability
        top_recommendations = top_recommendations[top_recommendations['selection_probability'] > 0]

        all_recommendations.append(top_recommendations)

    # Combine all recommendations
    if all_recommendations:
        recommendations_df = pd.concat(all_recommendations)

        print(f"\nGenerated recommendations for {len(customers)} customers")

        # Display sample recommendations for a few customers
        sample_customers = recommendations_df['customer_id'].unique()[:3]
        for customer in sample_customers:
            print(f"\nRecommendations for customer {customer}:")
            print(recommendations_df[recommendations_df['customer_id'] == customer])

        return recommendations_df
    else:
        print("No recommendations generated")
        return pd.DataFrame()

# Generate customer recommendations
customer_recommendations = generate_customer_recommendations(predictions)


In [ ]:
customer_recommendations.to_csv('customer_recommendations.csv', index=False)

In [ ]:
def run_sku_forecasting_pipeline(df, cutoff_date=None, prediction_window=14):
    """
    Run the complete SKU-level demand forecasting pipeline
    """
    # If no cutoff date is provided, use the latest date in the dataset
    if cutoff_date is None:
        cutoff_date = df['order_date'].max() - pd.Timedelta(days=prediction_window)

    print(f"Running forecast with cutoff date: {cutoff_date}")

    # Only use data prior to cutoff date for training
    historical_data = df[df['order_date'] <= cutoff_date].copy()

    # Creating features
    print("\nEngineering features...")
    cp_features = create_customer_product_features(historical_data)
    features_with_customer = add_customer_features(historical_data, cp_features)
    complete_features = add_product_features(historical_data, features_with_customer)
    final_features = add_interaction_features(complete_features)

    # Prepare training data
    print("\nPreparing training data...")
    training_data, feature_cols = prepare_training_data(df, final_features, cutoff_date, prediction_window)

    # Build models
    print("\nBuilding prediction models...")
    selection_model, selection_scaler = build_product_selection_model(training_data, feature_cols)
    quantity_model, quantity_scaler = build_quantity_prediction_model(training_data, feature_cols)

    # Make predictions
    print("\nMaking predictions...")
    predictions = predict_future_orders(
        final_features, selection_model, quantity_model,
        selection_scaler, quantity_scaler, feature_cols
    )

    # Aggregate demand for inventory planning
    print("\nAggregating demand for inventory planning...")
    product_demand = aggregate_demand_by_product(predictions)

    # Generate customer recommendations
    print("\nGenerating customer recommendations...")
    customer_recommendations = generate_customer_recommendations(predictions)

    return {
        'predictions': predictions,
        'product_demand': product_demand,
        'customer_recommendations': customer_recommendations,
        'models': {
            'selection_model': selection_model,
            'quantity_model': quantity_model,
            'selection_scaler': selection_scaler,
            'quantity_scaler': quantity_scaler
        }
    }


results = run_sku_forecasting_pipeline(merged)


In [ ]:
def calculate_inventory_requirements(predictions_df, product_demand_df, current_stock_df=None):
    """
    Calculate inventory requirements for next 14 days.
    """
    # Generate future date range
    start_date = pd.to_datetime("2025-04-15")  # Use current date from context
    date_range = pd.date_range(start=start_date, periods=14, name='order_date')

    # Create full product-date grid
    products = predictions_df['item_name'].unique()
    full_grid = pd.MultiIndex.from_product([products, date_range], names=['item_name', 'order_date']).to_frame(index=False)

    # Merge predictions with date grid
    predictions_df = full_grid.merge(predictions_df, on=['item_name'], how='left')
    predictions_df['predicted_quantity'] = predictions_df['predicted_quantity'].fillna(0)

    # Rest of original logic remains unchanged
    daily_demand = predictions_df.groupby(['item_name', 'order_date'])['predicted_quantity'].sum().unstack().fillna(0)

    projection = daily_demand.sum(axis=1).to_frame(name='14_day_demand')

    if current_stock_df is not None:
        projection = projection.merge(current_stock_df,
                                     left_index=True,
                                     right_on='item_name',
                                     how='left')
        projection['current_stock'] = projection['current_stock'].fillna(0)
    else:
        projection['current_stock'] = 0

    projection['safety_stock'] = (projection['14_day_demand'] * 0.2).round().astype(int) + 10

    projection['recommended_order'] = np.where(
        projection['current_stock'] < projection['14_day_demand'] + projection['safety_stock'],
        (projection['14_day_demand'] + projection['safety_stock'] - projection['current_stock']).clip(lower=0),
        0
    )

    return projection.reset_index()[['item_name', '14_day_demand',
                                    'current_stock', 'safety_stock',
                                    'recommended_order']]

In [ ]:
# Example: current_stock is a DataFrame with columns ['item_name', 'current_stock']
inventory_plan = calculate_inventory_requirements(
    predictions_df=predictions[predictions['will_order'] == 1][['item_name', 'predicted_quantity']],
    product_demand_df=None,  # Not needed if using predictions directly

)

In [ ]:
print(inventory_plan.head(10))  # Shows the top 10 products needing replenishment


In [ ]:
inventory_plan.to_csv('inventory_plan.csv', index=False)